In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd

# Veri setini yükledik.
data = pd.read_csv('/kaggle/input/diabetes-data-set/diabetes.csv')

# Bağımsız değişkenleri ve hedef değişkeni ayırdık.
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

# Önemli özellikleri seçtik.(Korelasyon haritasına göre).
important_features = ["Glucose", "BMI", "Age", "Pregnancies"]

X_important = X[important_features]

# Veriyi train ve test kümelerine ayırdık.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_important, y, test_size=0.25, random_state=42)

# Hiperparametre optimizasyonu
param_grid = {
    'n_estimators': [50, 100, 200, 300], # Ağaç sayısı
    'max_depth': [None, 5, 10, 20],            # Ağacın maksimum derinliği
    'min_samples_split': [2, 5, 10],           # Dallanma için minimum örnek sayısı
    'min_samples_leaf': [1, 2, 4],              # Yapraklar için minimum örnek sayısı
  }

rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# En iyi hiperparametreler ve en iyi doğruluk değeri
print("En iyi hiperparametreler:", grid_search.best_params_)
print("En iyi doğruluk değeri (Doğrulama Kümesi):", grid_search.best_score_)

# Test veri kümesi üzerinde tahmin yaptık
y_pred = grid_search.predict(X_test)

# Doğruluk değerini hesapla
accuracy = accuracy_score(y_test, y_pred)
print("Model Doğruluğu (Test Verisi):", accuracy)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Özellikler arasındaki korelasyon matrisini hesaplayalım
correlation_matrix = data.corr()

# Korelasyon analizi grafiğini çizelim
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, square=True, cbar_kws={"shrink": 0.8, "aspect": 30})
plt.title("Özellikler Arasındaki Korelasyon Matrisi", fontsize=18, fontweight='bold', pad=20)
plt.xticks(fontsize=12, rotation=45, ha='right')
plt.yticks(fontsize=12)
plt.xlabel('Özellikler', fontsize=14, labelpad=10)
plt.ylabel('Özellikler', fontsize=14, labelpad=10)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Test veri kümesi üzerinde olasılık değerlerini aldık.
y_prob = grid_search.predict_proba(X_test)[:, 1]

# ROC eğrisini çizelim
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR) (Sensitivity)')
plt.title('ROC Curve - Random Forest')
plt.legend()
plt.grid(True)
plt.show()

# AUC (Alan Altındaki Eğri) değeri
auc_score = roc_auc_score(y_test, y_prob)
print("AUC Değeri:", auc_score)

In [ ]:
from sklearn.metrics import precision_recall_curve

# Precision-Recall eğrisini hesaplayalım
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)

# Precision-Recall eğrisi
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall (Geri Çağırma)')
plt.ylabel('Precision (Hassasiyet)')
plt.title('Precision-Recall Eğrisi')
plt.grid(True)
plt.show()

In [ ]:
# Tahmin dağılımı grafiğini çizelim
plt.figure(figsize=(10, 6))
sns.histplot(data=y_prob, bins=50, kde=True, color='mediumorchid', alpha=0.7)
plt.axvline(x=0.5, color='red', linestyle='--', label='Sınıf Sınırı (0.5)')
plt.xlabel('Olasılık Değeri', fontsize=12)
plt.ylabel('Frekans', fontsize=12)
plt.title('Tahmin Dağılımı', fontsize=16, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
# Modelin eğitilmiş haliyle özellik önem sıralamasını alalım
feature_importances = grid_search.best_estimator_.feature_importances_

# Özellik önem sıralamasını bir DataFrame'e dönüştürelim
feature_importance_df = pd.DataFrame({'Features': important_features, 'Importance': feature_importances})

# Özellikleri öneme göre sıralayalım
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Özellik Önem Sıralaması Grafiğini çizelim
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Features', data=feature_importance_df, palette='coolwarm')
plt.title('Özellik Önem Sıralaması')
plt.xlabel('Önem Derecesi (%)')
plt.ylabel('Özellikler')
plt.xlim(0, 1)
plt.xticks(rotation=0)
for index, value in enumerate(feature_importance_df['Importance']):
    plt.text(value, index, f'{value*100:.2f}%', ha="left", va="center", color="white", fontweight='bold')
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve
import numpy as np

# En iyi modeli alalım
best_rf_model = grid_search.best_estimator_

# Öğrenme eğrisini çizelim
train_sizes, train_scores, test_scores = learning_curve(best_rf_model, X_train, y_train, cv=5, train_sizes=np.linspace(0.1, 1.0, 10))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Eğitim Performansı')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')

plt.plot(train_sizes, test_scores_mean, 'o-', color='orange', label='Test Performansı')
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.2, color='orange')
plt.xlabel('Eğitim Veri Kümesi Boyutu', fontsize=12)
plt.ylabel('Performans', fontsize=12)
plt.title('Öğrenme Eğrisi', fontsize=16, fontweight='bold')
plt.legend(loc='best')
plt.grid(True)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Başlık tanımlama
title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'black'}

# Seaborn ile özelliklerin dağılımını çizelim
plt.figure(figsize=(12, 8))

# Renk paleti tanımlama
color_palette = sns.color_palette("pastel")

for i, feature in enumerate(important_features, 1):
    plt.subplot(2, 2, i)
    sns.histplot(data[feature], bins=20, kde=True, color=color_palette[i-1], alpha=0.7)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Frekans', fontsize=12)
    plt.title(f'{feature} Dağılımı', fontsize=14, fontweight='bold')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

plt.suptitle('Önemli Özelliklerin Dağılımı', **title_font)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()